In [1]:
!pip install neo4j

In [2]:
from neo4j import GraphDatabase
import csv

class KnowledgeGraphBuilder:
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()

    def create_relationship(self, session, relationship_type, start_id, end_id, bidirectional=True):
        query = (
            f'MERGE (s:Company {{id: "{start_id}"}})'
            f'MERGE (o:Company {{id: "{end_id}"}})'
            f'MERGE (s)-[:{relationship_type}]->(o)'
        )
        if bidirectional:
            query += f'MERGE (o)-[:{relationship_type}]->(s)'
        session.run(query)

    def create_knowledge_graph(self):
        with self._driver.session() as session:
            # Read data from CSV files and create relationships
            csv_files = [
                "hidy.relationships.supply.csv",
                "hidy.relationships.same_industry.csv",
                "hidy.relationships.invest.csv",
                "hidy.relationships.dispute.csv",
                "hidy.relationships.cooperate.csv",
                "hidy.relationships.compete.csv",
            ]

            for file in csv_files:
                relationship_type = file.split(".")[2]
                with open(f"/Users/wufei/Downloads/dsaa5002_project/KnowledgeGraph/{file}", "r", encoding="utf-8") as csvfile:
                    csv_reader = csv.reader(csvfile)
                    next(csv_reader)  # Skip header row
                    for row in csv_reader:
                        # Set bidirectional to False for "invest" and "supply"
                        bidirectional = relationship_type not in ["invest", "supply"]
                        self.create_relationship(
                            session, relationship_type, row[0], row[1], bidirectional=bidirectional
                        )

if __name__ == "__main__":
    uri = "bolt://localhost:7687"  # Your Neo4j connection URI
    user = "neo4j"
    password = "lei'sproject"

    builder = KnowledgeGraphBuilder(uri, user, password)
    builder.create_knowledge_graph()
    builder.close()
